In [1]:
# isozyme.csv ... three columns in this order: index of mol, smiles, half-life

In [2]:
# isozyme_data-split.csv ... same three colums, but mols and their indexes are shuffeled
# after using ScaffoldSplitter from deepchem on the list of smiles
# (how to preserve the pairing between smiles and its index?)
# data-splits are ["train", "test"]

In [3]:
# orig 3A4 csv file from ChEMBL -> only mols w/ "Standard Type" == "T1/2" (must be uppercase*)
# and only "Standard Value" column -> 3A4.csv w/ 70 mols
# * ... mols w/ "t1/2" don't have a "Standard Value"

In [4]:
# orig RLM csv file from PubChem -> remove invalid smiles (how?*) -> RLM.csv w/ 2524 mols
# * ... one way would be to convert them to jazzy molecule features
# and remove mols with empty dict

In [5]:
# orig HLC csv file from PubChem -> convert ">30" to 30 -> HLC.csv w/ 189 mols

In [1]:
import sys
import numpy as np
import pandas as pd
import deepchem as dc
from project_resources.import_utils import NotebookFinder
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import *

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
C:\Users\Lukas\anaconda3\envs\test_two\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (C:\Users\Lukas\anaconda3\envs\test_two\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


importing Jupyter notebook from C:\Users\Lukas\Documents\datacytochromy\project_resources\cytochrome_P450.ipynb


In [97]:
isozymes = ["3A4", "RLM", "HLC"]
rel_paths = {
    "3A4_source": r"project_resources/ChEMBL_3A4.csv",
    "3A4_sep": ";",
    "3A4": r"project_resources/3A4.csv",
    "3A4_train": r"project_resources/data_splits/3A4_train.csv",
    "3A4_test": r"project_resources/data_splits/3A4_test.csv",

    "RLM_source": r"project_resources/AID_1508591_datatable_all.csv",
    "RLM_sep": ",",
    "RLM": r"project_resources/RLM.csv",
    "RLM_train": r"project_resources/data_splits/RLM_train.csv",
    "RLM_test": r"project_resources/data_splits/RLM_test.csv",

    "HLC_source": r"project_resources/AID_1508603_datatable_all.csv",
    "HLC_sep": ",",
    "HLC": r"project_resources/HLC.csv",
    "HLC_train": r"project_resources/data_splits/HLC_train.csv",
    "HLC_test": r"project_resources/data_splits/HLC_test.csv"
}
smiles = {}
halflife = {}
smiles_as_index = {}  # structure: {isozyme: {"smi": (idx, halflife),...}}
split_smiles = {}

In [98]:
# create and/or load csv files
for isozyme in isozymes:
    csv_data_formatting(rel_paths[f"{isozyme}_source"], isozyme, sep=rel_paths[f"{isozyme}_sep"])

3A4.csv already exists in dir
RLM.csv already exists in dir
HLC.csv already exists in dir


In [99]:
# read smiles, their idxs and corresponding half-life as tuple(idx, smiles, half-life)
for isozyme in isozymes:
    df = pd.read_csv(rel_paths[isozyme])
    index = df["mol_idx"]
    smiles[isozyme] = []
    smiles[isozyme] = list(df["smiles"])
    df_halflife = df["half-life"]
    smiles_as_index[isozyme] = {}
    halflife[isozyme] = []
    for idx, smi, val in zip(index, smiles[isozyme], df_halflife):
        smiles_as_index[isozyme][smi] = (idx, val)
        halflife[isozyme].append(val)
    first_smiles = smiles[isozyme][0]
    print(f"{smiles[isozyme][:5]}\n{halflife[isozyme][:5]}\n{smiles_as_index[isozyme][first_smiles]}")

['COc1ccc2[nH]cc(CCNC(C)=O)c2c1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3nnc(Cc4ccc(F)cc4F)cc32)[C@@H](CN2Cc3c(F)cccc3C2=O)CN1', 'O=c1[nH]c2ccccc2n1C1CCN(CCCC(c2ccc(F)cc2)c2ccc(F)cc2)CC1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3ncc([C@@H](O)c4ccc(F)cc4)cc32)[C@@H](CN2[C@H](C)COC[C@H]2C)CN1', 'C[C@@H]1CN(CC(=O)N2CC(C)(C)c3ncc(Cc4ccc(F)cc4)cc32)[C@@H](CN2Cc3ccccc3C2=O)CN1']
[0.3333, 1.167, 0.045, 0.4667, 0.75]
(1, 0.3333)
['CCCCOC1=CC=C(C=C1)CC(=O)NO', 'COC1=C(C=C(C=C1)CCN2C(=CC(=O)NC2=S)N)OC', 'COC1=C(C=C(C=C1)Cl)C(=O)NC2=CC=C(C=C2)NC(=O)C3=CC=CO3', 'CC(C)(C)N1C2=NC=NC(=C2C(=N1)C3=CC=C(C=C3)Cl)N', 'CN1C2=CC=CC=C2C(=O)C3=C1N=C(N(C3=O)C4=CC=CC=C4)C5CCCCC5']
[30.0, 30.0, 30.0, 30.0, 30.0]
(1, 30.0)
['CC1=CC2=CC(=NC(=C2O1)NC3=CC(=CC=C3)F)C4=C(ON=C4C)C', 'CN(CC1=CC(=CC=C1)F)C2CCOC3=CC=CC=C23', 'C1COC2=C1C(=CN=C2NC3=CN=CN=C3)C4=CC(=CC=C4)F', 'C1CN(C2=C1C=CC=N2)C(=O)C3=CC=C(C=C3)C4=NC=C5N4C=C(N=C5)C6=CC(=CC=C6)F', 'CC1=CC=C(C=C1)NC(=O)C2=C(N3C=CSC3=N2)C4=CC=CC(=C4)C#N']
[8.2, 8.6, 9.6, 17.3, 17.4]
(1, 8.2)


In [100]:
# train-test split with deepchem's ScaffoldSplitter
for isozyme in isozymes:
    identifiers = smiles[isozyme]
    Xs = np.zeros(len(identifiers))
    dataset = dc.data.DiskDataset.from_numpy(X=Xs, ids=identifiers)
    scaffoldsplitter = dc.splits.ScaffoldSplitter()
    train, test = scaffoldsplitter.train_test_split(dataset)
    split_smiles[isozyme] = {}
    split_smiles[isozyme]["train"] = train.ids.tolist()
    split_smiles[isozyme]["test"] = test.ids.tolist()
    print(f"{isozyme}\ntrain:\n{split_smiles[isozyme]['train'][:5]}\ntest:\n{split_smiles[isozyme]['test'][:5]}\n")

3A4
train:
['COc1cccc([C@@H](CO)NC(=O)[C@@H](C)N2Cc3ccc(-c4nc(NC5CCOCC5)ncc4Cl)cc3C2=O)c1', 'COc1cc(F)cc([C@H](NC(=O)CN2Cc3ccc(-c4nc(NC5CCOCC5)ncc4Cl)cc3C2=O)C(C)O)c1', 'COc1cc(F)cc([C@@H](CO)NC(=O)[C@@H](C)N2Cc3ccc(-c4ccnc(NC5CCOCC5)n4)cc3C2=O)c1', 'Cc1cccc([C@@H](CO)NC(=O)CN2Cc3ccc(-c4nc(NC5CCOCC5)ncc4Cl)cc3C2=O)c1', 'Cc1cccc([C@@H](CO)NC(=O)[C@@H](C)N2Cc3ccc(-c4nc(NC5CCOCC5)ncc4Cl)cc3C2=O)c1']
test:
['O=C1CCC(N2C(=O)c3cccc(NCCOCCOCCNC(=O)c4ccc5c(c4)nc(Nc4cccc(Cl)c4)c4ccncc45)c3C2=O)C(=O)N1', 'O=C(CCCN1CCC(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F)cc1', 'O=C(NC1CCN(CCc2c[nH]c3ccccc23)CC1)c1ccccc1', 'Cc1ncsc1-c1ccc([C@H](CC(=O)NCCCCCCNC(=O)COc2c(-c3csc(N4CCOCC4)n3)ccc(F)c2F)NC(=O)[C@@H]2C[C@@H](O)CN2C(=O)[C@@H](NC(=O)C2(F)CC2)C(C)(C)C)cc1', 'COc1ccc(CCN2CCC(Nc3nc4ccccc4n3Cc3ccc(F)cc3)CC2)cc1']

RLM
train:
['CS(=O)(=O)C1=CC=CC=C1C2=CSC(=N2)N3CCC(CC3)C(=O)N', 'C1CN(CCC1C(=O)N)C2=NC(=CS2)C3=CC(=CC=C3)N', 'CC(=O)NC1=CC=C(C=C1)C2=CSC(=N2)N3CCC(CC3)C(=O)N', 'CN(C)C(=O)C1=CC=CC(=C1)C2=CSC(=N2)N3CCC(C

In [101]:
for isozyme in isozymes:
    for split in ["train", "test"]:
        split_df = pd.DataFrame()
        split_indexes = []
        split_halflifes = []
        isozyme_split_smiles = split_smiles[isozyme][split]

        # get the index and half-life values for each smiles in data split
        for smi in isozyme_split_smiles:
            smi_idx = smiles_as_index[isozyme][smi][0]  # numerical index of smiles
            split_indexes.append(smi_idx)
            mol_halflife = smiles_as_index[isozyme][smi][1]  # half-life value for the specific molecule
            split_halflifes.append(mol_halflife)

        split_df["index"] = split_indexes
        split_df["smiles"] = isozyme_split_smiles
        split_df["half-life"] = split_halflifes
        split_df.to_csv(rel_paths[f"{isozyme}_{split}"], index=False)